In [ ]:
from datetime import datetime
import re

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoMinorLocator

In [ ]:
# log_file = '/home/universal/Downloads/tmp.log'
# log_file = '/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/demo_scans/tmp_2.log'
log_file = '/home/ovoinov/remotes/root.thanos/home/universal/Downloads/dev.sk_robot_rgbd_data/experiments/logs/scanning/22_02_28_cameras.log'

In [ ]:
def is_logger_msg(line):
    return (
        line.startswith('2022-')
        and line.partition(':   ')[1] != ''
    )


def parse_line(l):
    try:
        if not is_logger_msg(l):
            return dict(time=None, context='', msg='')
        meta, _, msg = l.partition(':   ')
        msg = msg[:-1]
        date, time, loglevel, context = meta.split()
        time = datetime.strptime(f'{date} {time}000', '%Y-%m-%d %H:%M:%S,%f')
        return dict(time=time, context=context, msg=msg)
    except:
        print(l)
        raise

In [ ]:
scene = 'golden_bust'

lines = open(log_file).readlines()
start_i = -1
while parse_line(lines[start_i])['msg'] != f'Scan with cameras {scene}':
    start_i -= 1
    
end_i = -1
while parse_line(lines[end_i])['msg'] != f'Scan with cameras {scene} DONE':
    end_i -= 1
    
lines = lines[start_i: end_i + 1]

lines = list(filter(is_logger_msg, lines))

In [ ]:
for l in lines:
    data = parse_line(l)
    if data['msg'] == 'Move robot':
        start = data['time']
        break

In [ ]:
cameras = ['kinect_v2', 'phone_left', 'phone_right', 'real_sense', 'tis_left', 'tis_right']

events = [
    '^Move robot$',
    '^Move robot DONE$',
    '^Wait after Move robot$',
    '^Wait after Move robot DONE$',
    '^Close Kinect IR$',
    '^Save phone_left \(1\) depth$',
    '^Save phone_right \(2\) depth$',
    '^Setup light.*$(?<!DONE)',
    '^Setup light.* DONE$',
    '^Open Kinect IR$',
    '^Save Kinect depth$',
]

for camera in cameras:
    events.append(f'^Setup {camera}.*$(?<!DONE)')
    events.append(f'^Setup {camera}.* DONE$')
    events.append(f'{camera}: Taking image')
    events.append(f'{camera}: Taking image DONE')
    events.append(f'^Wait after Setup light .* #{camera}$(?<!DONE)')
    events.append(f'^Wait after Setup light .* #{camera} DONE$')

In [ ]:
times = {event: [] for event in events}
lights = set()

for l in lines:
    data = parse_line(l)
    context = data['context']
    msg = data['msg']
    time = (data['time'] - start).total_seconds()
    if time < 0:
        continue
    if context == 'ScanHelper':
        for event in events:
            if re.match(event, msg):
                times[event].append(time)
        if msg.startswith('Setup light'):
            lights.add(msg)

    if context in cameras:
        if re.match('^Taking image$(?<!DONE)', msg):
            times[f'{context}: Taking image'].append(time)
        if re.match('^Taking image DONE$', msg):
            times[f'{context}: Taking image DONE'].append(time)

times = {k: np.array(v) for (k, v) in times.items()}
lights_n = len(lights)

In [ ]:
y = 0
figure = plt.figure(figsize=(30, 15))

plt.barh(
    y, label='move',
    left=times['^Move robot$'], width=times['^Move robot DONE$'] - times['^Move robot$'],
    color='tab:blue'
)
plt.barh(
    y, label='shake',
    left=times['^Wait after Move robot$'], width=times['^Wait after Move robot DONE$'] - times['^Wait after Move robot$'],
    alpha=.3, color='tab:blue'
)

y = -1
# plt.barh(
#     y, label='close ir',
#     left=times['^Close Kinect IR$'], width=times['^Save phone.* \(1\) depth$'] - times['^Close Kinect IR$'],
#     color='tab:brown', alpha=1
# )
plt.barh(
    y, label='open ir',
    left=times['^Open Kinect IR$'], width=times['^Save Kinect depth$'] - times['^Open Kinect IR$'],
    color='tab:brown', alpha=.3
)
plt.barh(
    y, label='phone 1 depth',
    left=times['^Save phone_left \(1\) depth$'], width=times['^Save phone_right \(2\) depth$'] - times['^Save phone_left \(1\) depth$'],
    color='tab:red', hatch='/'
)
plt.barh(
    y, label='phone 2 depth',
    left=times['^Save phone_right \(2\) depth$'], width=times['^Open Kinect IR$'] - times['^Save phone_right \(2\) depth$'],
    color='tab:red', hatch='\\'
)

y = -2
plt.barh(
    y, label='light',
    left=times['^Setup light.*$(?<!DONE)'], width=times['^Setup light.* DONE$'] - times['^Setup light.*$(?<!DONE)'],
    color='yellow',
)

kwargs = {
    'kinect_v2': {'color': 'tab:blue'},
    'phone_left': {'color': 'tab:orange'},
    'phone_right': {'color': 'tab:green'},
    'real_sense': {'color': 'tab:red'},
    'tis_left': {'color': 'tab:purple'},
    'tis_right': {'color': 'tab:pink'},
}
for camera in cameras:
    y -= 1
    if camera not in {'tis_left', 'tis_right', 'kinect_v2'}:
        plt.barh(
            y,
            left=times[f'^Setup {camera}.*$(?<!DONE)'], width=times[f'^Setup {camera}.* DONE$'] - times[f'^Setup {camera}.*$(?<!DONE)'],
            hatch='x', **kwargs[camera]
        )
    plt.barh(
        y,
        left=times[f'^Wait after Setup light .* #{camera} DONE$'], width=times[f'^Wait after Setup light .* #{camera}$(?<!DONE)'] - times[f'^Wait after Setup light .* #{camera} DONE$'],
        alpha=.3, **kwargs[camera]
    )
    plt.barh(
        y, label=camera,
        left=times[f'{camera}: Taking image'], width=times[f'{camera}: Taking image DONE'] - times[f'{camera}: Taking image'],
        **kwargs[camera]
    )


plt.grid()
plt.legend()
ax = plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.xaxis.set_minor_locator(MultipleLocator(.1))
ax.tick_params(which='minor', length=4)
plt.xlim(0, 1);
plt.close()

In [ ]:
plt.sca(ax)
i = 5
x0 = times['^Move robot$'][i] - 1
x1 = times['^Move robot$'][i + 1] + 1
plt.xlim(x0, x1);

In [ ]:
plt.sca(ax)
i = 5
x0 = times['^Move robot$'][i] - 1
x1 = times['^Move robot$'][i + 1] + 1
plt.xlim(x0, x1);